In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Data Preprocessing Pipeline for Instagram Data
def read_and_clean_data(file_path, output_file):
    """
    Reads and preprocesses Instagram comment data from CSV and saves a single output file.

    Args:
    - file_path (str): Path to input CSV file.
    - output_file (str): File path for saving the output CSV.

    Returns:
    - pd.DataFrame: Cleaned comments DataFrame.
    """
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading file: {e}")
        return None
    
    # Drop unnecessary columns and rename
    columns_to_drop = ['profilePictureUrl', 'username', 'profileUrl', 
                       'commentId', 'ownerId', 'timestamp']
    df.drop(columns=columns_to_drop, errors='ignore', inplace=True)
    
    df.columns = ['comments', 'comment_like', 'reply_count', 
                  'comment_date', 'post_urls']
    
    # Date processing
    df['comment_date'] = pd.to_datetime(df['comment_date'], errors='coerce')
    df[['comment_year', 'comment_month']] = (
        df['comment_date'].apply(lambda x: pd.Series([x.year, x.month]))
    )
    df.drop(columns='comment_date', inplace=True)
    pd.options.display.float_format = '{:.0f}'.format
    
    # Filter valid URLs and extract short_code
    df = df[df['post_urls'].apply(lambda x: isinstance(x, str))]
    df['short_code'] = df['post_urls'].str.extract(r'/p/([^/?]+)/')
    
    # Reorder columns
    df = df[['post_urls', 'short_code', 'comments', 
             'comment_like', 'reply_count', 
             'comment_year', 'comment_month']]
    
    df.index = np.arange(1, len(df) + 1)
    
    # Save output as a single file
    df.to_csv(output_file, index=False)
    print(f"Saved file: {output_file}")
    
    return df

def process_post_details(input_file, output_file):
    """
    Processes Instagram post details from CSV and saves the result.

    Args:
    - input_file (str): Path to input CSV.
    - output_file (str): Path to save output CSV.

    Returns:
    - pd.DataFrame: Processed post details DataFrame.
    """
    try:
        post_details = pd.read_csv(input_file)
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

    # Select and rename required columns
    required_columns = ['caption', 'display_url', 
                        'like_count', 'post_date', 
                        'short_code']
    post_details = post_details[required_columns]
    post_details.columns = ['caption', 'display_url', 
                            'likes', 'post_date', 
                            'short_code']


    # Add proper indexing
    post_details.index = np.arange(1, len(post_details) + 1)

    # Convert 'post_date' to datetime and extract components
    post_details['post_date'] = pd.to_datetime(post_details['post_date'], errors='coerce')
    post_details['post_year'] = post_details['post_date'].dt.year
    post_details['post_month'] = post_details['post_date'].dt.month
    
    # Drop the original date column
    post_details.drop(columns=['post_date'], inplace=True, errors='ignore')
    pd.options.display.float_format = '{:.0f}'.format
    
    # Save output
    post_details.to_csv(output_file, index=False)
    print(f"Post details saved to {output_file}")
    
    return post_details


def merging_files(comments_df, posts_df, output_file):
    """
    Merges comment and post details DataFrames on 'short_code' and saves the output.

    Args:
    - comments_df (pd.DataFrame): Comments DataFrame.
    - posts_df (pd.DataFrame): Posts DataFrame.
    - output_file (str): Path to save merged CSV.

    Returns:
    - pd.DataFrame: Merged DataFrame.
    """
    if isinstance(comments_df, str):
        comments_df = pd.read_csv(comments_df)
    if isinstance(posts_df, str):
        posts_df = pd.read_csv(posts_df)

    # Perform an inner merge on short_code
    merged_df = pd.merge(comments_df, posts_df, on='short_code', how='inner')
    
    # Reorder columns for consistency
    merged_df = merged_df[['short_code', 'caption',  
                           'likes', 'post_year', 'post_month',
                           'comments', 
                           'comment_like', 'reply_count', 
                           'comment_year', 'comment_month']]
    
    pd.options.display.float_format = '{:.0f}'.format
    merged_df.index = np.arange(1, len(merged_df) + 1)
    
    # Save output
    merged_df.to_csv(output_file, index=False)
    print(f"Merged data saved to {output_file}")
    merged_df.index = np.arange(1, len(merged_df) + 1)
    
    return merged_df


def append_files(input_files, output_file):
    """
    Appends multiple CSV files into a single CSV file.

    Args:
    - input_files (list of str): List of file paths to append.
    - output_file (str): Path to save the appended CSV.

    Returns:
    - pd.DataFrame: Combined DataFrame.
    """
    all_dfs = []

    for file in input_files:
        try:
            df = pd.read_csv(file)
            all_dfs.append(df)
            print(f"Appended: {file}")
        except Exception as e:
            print(f"Error reading {file}: {e}")
    
    # Concatenate all DataFrames
    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_df.index = np.arange(1, len(combined_df) + 1)
    
    # Save combined CSV
    combined_df.to_csv(output_file, index=False)
    print(f"Combined CSV saved to: {output_file}")

    return combined_df

In [ ]:
# For Comments data 
input_file = r"C:\Users\hirshikesh\Desktop\Capstone Project\phathom_Insta_Costco_results.csv"
output_file = r"C:\Users\hirshikesh\Desktop\Capstone Project\InstaCostco_Part1.csv"

Data = read_and_clean_data(input_file, output_file)

In [ ]:
Data

In [ ]:
# For Posts details
post_file = r"C:\Users\hirshikesh\Desktop\Capstone Project\instagram-post-details.csv"
post_output = r"C:\Users\hirshikesh\Desktop\Capstone Project\post_details.csv"

post_details = process_post_details(post_file, post_output)

In [ ]:
post_details

In [ ]:
# Final Merged Output (Comments data and Post details)
comments_df = r"C:\Users\hirshikesh\Desktop\Capstone Project\InstaCostco_Part1.csv"
posts_df = r"C:\Users\hirshikesh\Desktop\Capstone Project\post_details.csv"
final_output = r"C:\Users\hirshikesh\Desktop\Capstone Project\Final_Part_1.csv"

final_part1 = merging_files(comments_df, posts_df, final_output)

In [ ]:
final_part1

In [ ]:
# Filter and display rows where 'short_code' is a float (likely NaN)
missing_urls = final_part1[final_part1['short_code'].apply(lambda x: isinstance(x, float))]
print(missing_urls)

In [ ]:
# For Posts details
post_file_1 = r"C:\Users\hirshikesh\Desktop\Capstone Project\remaining_instagram-post-details_1.csv"
post_output_1 = r"C:\Users\hirshikesh\Desktop\Capstone Project\remaining_post_details.csv"

post_details1 = process_post_details(post_file_1, post_output_1)

In [ ]:
post_details1

In [ ]:
# Final Merged Output (Comments data and Post details)
comments_df1 = r"C:\Users\hirshikesh\Desktop\Capstone Project\Apify_Costco_Instagram_results.csv"
posts_df1 = r"C:\Users\hirshikesh\Desktop\Capstone Project\remaining_post_details.csv"
final_output1 = r"C:\Users\hirshikesh\Desktop\Capstone Project\Final_Part_2.csv"

final_part2 = merging_files(comments_df1, posts_df1, final_output1)

In [ ]:
final_part2

In [ ]:
# Filter and display rows where 'short_code' is a float (likely NaN)
missing_urls = final_part2[final_part2['short_code'].apply(lambda x: isinstance(x, float))]
print(missing_urls)

In [ ]:
# Lets append the final preprocessed parts (Final_Part1 and Final_Part2)
final_part_1 = r"C:\Users\hirshikesh\Desktop\Capstone Project\Final_Part_1.csv"
final_part_2 = r"C:\Users\hirshikesh\Desktop\Capstone Project\Final_Part_2.csv"

# Output combined file
final_combined_output = r"C:\Users\hirshikesh\Desktop\Capstone Project\cleaned_costco_data_initial.csv"

# Call the append function
initial_cleaned_data = append_files(
    input_files=[final_part_1, final_part_2], 
    output_file=final_combined_output
)

In [ ]:
initial_cleaned_data

In [ ]:
# Filter and display rows where 'short_code' is a float (likely NaN)
missing_urls = initial_cleaned_data[initial_cleaned_data['short_code'].apply(lambda x: isinstance(x, float))]
print(missing_urls)